In [1]:
import pandas as pd
import os
import datetime
from pm4py.objects.conversion.log.versions import to_dataframe
import warnings
warnings.filterwarnings('ignore')
from pm4py.objects.log.importer import xes
from pm4py.objects.log.importer.xes import importer as xes_importer
import pytz

In [2]:
wd = os.getcwd()
"""
get all filenames in working directory
then collect log containing folders
get log files paths and collect logs

"""
logs_paths = []
for file in os.listdir(wd):
    if file.endswith(".xes_"):
        for f in os.listdir(file):
            if f.endswith(".xes"):
                logs_paths.append(os.path.join(wd+'\\'+file, f))
print(logs_paths)

['C:\\PM contest\\Common challenge\\DomesticDeclarations.xes_\\DomesticDeclarations.xes', 'C:\\PM contest\\Common challenge\\InternationalDeclarations.xes_\\InternationalDeclarations.xes', 'C:\\PM contest\\Common challenge\\PermitLog.xes_\\PermitLog.xes', 'C:\\PM contest\\Common challenge\\PrepaidTravelCost.xes_\\PrepaidTravelCost.xes', 'C:\\PM contest\\Common challenge\\RequestForPayment.xes_\\RequestForPayment.xes']


### Таблицы

In [3]:
domestic_declarations = xes_importer.apply(wd + '\DomesticDeclarations.xes_\DomesticDeclarations.xes')

In [4]:
int_declarations = xes_importer.apply(wd + '\InternationalDeclarations.xes_\InternationalDeclarations.xes')

In [5]:
permit_log = xes_importer.apply(wd + '\PermitLog.xes_\PermitLog.xes')

In [6]:
prepaid_travel_cost = xes_importer.apply(wd + '\PrepaidTravelCost.xes_\PrepaidTravelCost.xes')

In [7]:
request_for_payment = xes_importer.apply(wd + '\RequestForPayment.xes_\RequestForPayment.xes')

In [9]:
permit_log = pd.read_csv('C://PM contest//Common challenge//PermitLog.xes_//compact_PermitLog.csv')
permit_log['time:timestamp']=permit_log['time:timestamp'].map(lambda x: datetime.datetime.fromisoformat(x))

In [ ]:
dfs_list = [
    domestic_declarations,int_declarations,permit_log,prepaid_travel_cost,request_for_payment
]

In [8]:
for dataframe in dfs_list:
    dataframe = to_dataframe(dataframe)

### Timestamp with timezone --> timestamp unified

In [10]:
"""
converts to UTC-format for unified timestamp format
:param: df - selected table
:param: time_column - column with the following time format %Y-%m-%d HH:MM:SS+0X:00
:return: converted to UTC-format timestmp without timezone
"""

def to_utc(df, time_column):
    utc = pytz.timezone('UTC')
    df[time_column] = df[time_column].map(lambda x: x.astimezone(utc).replace(tzinfo=None))

In [12]:
for dataframe in dfs_list:
    dataframe = to_utc(dataframe, 'time:timestamp')

# Questions

### 1. What is the throughput of a travel declaration from submission (or closing) to paying?

#### Preprocessing part

In [13]:
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [47]:
""" 
time delta between events for each event type its first mention is used
:param: df - selected table
:param: id_column - column where case id is stored
:param: concept_column - column where case name is stored
:param: eventlist - list where selected events are stored 
:param: time_column - column where timestamp in the format is stored timestamp(%Y-%m-%d HH:MM:SS)
:return: table with description statistics, 
         table with initial data: declaration ids, timedelta, timedelta in hours
"""

def events_time_delta(df, id_column, concept_column, eventlist, time_column):
    df=df.sort_values([id_column, time_column],ascending = True)
    throughoutput = df[df[concept_column].isin(set(eventlist))]
    throughoutput = throughoutput.drop_duplicates(subset = [id_column, concept_column], keep = 'first')
    tmp = pd.DataFrame(throughoutput.groupby([id_column]).apply(lambda x: list(x[time_column])))
    tmp = tmp.reset_index()
    tmp.columns = [id_column, 'timing']
    tmp['length'] = tmp['timing'].map(lambda x: len(x))
    tmp['throughoutput'] = None
    # the function does not count timedelta for events that are not finished
    tmp_selected = tmp[tmp['length']==2]
    tmp_selected['throughoutput'] = tmp_selected['timing'].map(lambda x: x[1]-x[0])
    tmp_selected['throughoutput_hours']=tmp_selected['throughoutput'].map(lambda x: x.total_seconds()/3600)
    tmp_selected = tmp_selected.drop(['length', 'timing'], axis =1)
    return pd.DataFrame(tmp_selected['throughoutput_hours'].describe()), tmp_selected

#### Exploration part (in dev)

In [60]:
t1, t2 = events_time_delta(int_declarations,
                          id_column='case:id',
                          concept_column='concept:name',
                          eventlist = ['Declaration SUBMITTED by EMPLOYEE','Payment Handled'],
                          time_column = 'time:timestamp')

In [61]:
t1

,throughoutput_hours
count,6187.000000
mean,350.597599
std,416.555390
min,26.096389
25%,156.879028
50%,245.887222
75%,407.536528
max,10298.707778


In [ ]:
int_declarations['year'] = int_declarations['time:timestamp'].map(lambda x: x.year)
domestic_declarations['year'] = domestic_declarations['time:timestamp'].map(lambda x: x.year)

In [ ]:
dd_18 = domestic_declarations[domestic_declarations['year']==2018]
int_18 = int_declarations[int_declarations['year']==2018]

In [ ]:
desc_table_dom, time_dom = throughput(dd_18, ['Declaration SUBMITTED by EMPLOYEE','Payment Handled'])

In [ ]:
desc_table_int, time_int = throughput(int_18, ['Declaration SUBMITTED by EMPLOYEE','Payment Handled'])

In [ ]:
time_int['label'] = 'international'
time_dom['label'] = 'domestic'
time_gen = pd.concat([time_int, time_dom])

In [ ]:
from math import log
time_int['log_time'] = time_int['throughoutput_hours'].map(lambda x: log(x))
time_dom['log_time'] = time_dom['throughoutput_hours'].map(lambda x: log(x))

In [ ]:
import plotly.figure_factory as ff
import numpy as np

In [ ]:
# данные для гистограм
x1 = time_int['throughoutput_hours'].to_list()
x2 = time_dom['throughoutput_hours'].to_list()
hist_data = [x1, x2]

# метки групп
group_labels = ['internatinal', 'domestic']

# кастомизация размера бинов
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)


#fig.show() у меня закомментировано, подыхает оперативка :)

In [ ]:
import plotly.offline as po

In [ ]:
po.plot(fig, filename = 'time_KDF_18.html')

In [ ]:
# данные для гистограм
x1 = time_int['log_time'].to_list()
x2 = time_dom['log_time'].to_list()
hist_data = [x1, x2]

# метки групп
group_labels = ['internatinal', 'domestic']

# кастомизация размера бинов
fig = ff.create_distplot(hist_data, group_labels, bin_size=.1)
#fig.show() у меня закомментировано, подыхает оперативка :)

po.plot(fig, filename = 'time_log_KDF_18.html')

In [ ]:
pd.DataFrame(time_int['log_time'].describe())

In [ ]:
pd.DataFrame(time_dom['log_time'].describe())

## 2. Is there are difference in throughput between national and international trips?

> На графике видно, что распределение может быть похоже на нормальное после преобразования. Вторая гипотеза - это то, что оно может быть би-модальное, иметь несколько мод (часто встречающихся значений). Так как перед нами на графиках функция плотности, по оси Y мы видим вероятность появления такого события в выборке, соответсвенно, где самые большие холмы - там наибольшая вероятность, таких событий больше всего в выборке. Чтобы понять, так это или нет, нужно провести тесты.

In [57]:
from scipy.stats import kstest,ks_2samp

In [58]:
import statsmodels.api as sm

#### Как получается КК (квантиль-квантиль) график:

![QQ](qq_plot.png)

In [ ]:
sm.qqplot(time_int['log_time'], line='s')

In [ ]:
sm.qqplot(time_dom['log_time'], line='s')

#### Тест Колмогорова-Смирнова на нормальное распределение

In [ ]:
print(kstest(time_dom['log_time'].to_list(), 'norm'))

In [ ]:
print(kstest(time_int['log_time'].to_list(), 'norm'))

> если p-value<0.05, то распределение не нормальное

#### Дискретный тест К-С на то, что нвблюдения соответсвуют одному распределению

In [ ]:
# p-value просто огромное, согласно тесту, наши наблюдения из одного распределения
ks_2samp(time_int['log_time'], time_dom['log_time'])

> <b>Инсайт:</b> за 2018 год обработка деклараций от сабмита работником до оплаты статистически не различается для деклараций на международные командировки и деклараций на внутренние командировки

In [ ]:
# DFG репрезентация 

In [ ]:
dd_18 = dd_18[dd_18['case:id'].isin(set(time_dom['case:id']))]

In [ ]:
dd_18.columns=['id', 'org:resource', 'concept:name', 'time:timestamp_old', 'org:role',
       'case:id', 'case:concept:name', 'case:BudgetNumber',
       'case:DeclarationNumber', 'case:Amount', 'year', 'timezone', 'time:timestamp']

In [ ]:
int_18.columns = ['id', 'org:resource', 'concept:name', 'time:timestamp_old', 'org:role',
       'case:Permit travel permit number', 'case:DeclarationNumber',
       'case:Amount', 'case:RequestedAmount', 'case:Permit TaskNumber',
       'case:Permit BudgetNumber', 'case:OriginalAmount',
       'case:Permit ProjectNumber', 'case:concept:name',
       'case:Permit OrganizationalEntity', 'case:travel permit number',
       'case:Permit RequestedBudget', 'case:id', 'case:Permit ID',
       'case:Permit id', 'case:BudgetNumber', 'case:Permit ActivityNumber',
       'case:AdjustedAmount', 'time:timestamp', 'timezone', 'year']

In [ ]:
from pm4py.objects.conversion.log import factory as conversion_factory

log_d18 = conversion_factory.apply(dd_18)

In [ ]:
from pm4py.algo.discovery.dfg import factory as dfg_factory

dfg_d18 = dfg_factory.apply(log_d18)

In [ ]:
from pm4py.visualization.dfg import factory as dfg_vis_factory

gviz = dfg_vis_factory.apply(dfg_d18, log=log_d18, variant="frequency")
dfg_vis_factory.view(gviz)

In [ ]:
from pm4py.algo.discovery.dfg import factory as dfg_factory
from pm4py.visualization.dfg import factory as dfg_vis_factory

dfg_d18 = dfg_factory.apply(log_d18, variant="performance")
gviz = dfg_vis_factory.apply(dfg_d18, log=log_d18, variant="performance")
dfg_vis_factory.view(gviz)

In [ ]:
log_int18 = conversion_factory.apply(int_18)
dfg_int18 = dfg_factory.apply(log_int18)
gviz = dfg_vis_factory.apply(dfg_int18, log=log_int18, variant="frequency")
dfg_vis_factory.view(gviz)

In [ ]:
dfg_int18 = dfg_factory.apply(log_int18, variant="performance")
gviz = dfg_vis_factory.apply(dfg_int18, log=log_int18, variant="performance")
dfg_vis_factory.view(gviz)